In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
data=pd.read_csv("bank-additional-full.csv")
data.rename(columns={'loan':'personal_loan','contact':'communication_means','month':'communication_month','day_of_week':'communication_day','fduration_of_call':'duration_of_call','pdays':'last_contacted','previous':'how_many_times_contacted','poutcome':'last_call_outcome','Result(y)':'Result (y)','emp.var.rate':'emp_var_rate'},inplace=True)
data.info()
data.head()
data.describe()
data.select_dtypes(include=['int64','float64']).columns
data.select_dtypes(include=['object']).columns

import sweetviz as sv # library for univariant analysis
my_report=sv.analyze(data) # passing the original dataframe
my_report.show_html() # arguments will generate to the library 

data1=data[['age', 'duration', 'campaign', 'last_contacted',
       'how_many_times_contacted', 'emp_var_rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed']]
data2=data[['job', 'marital', 'education', 'default', 'housing', 'personal_loan',
       'communication_means', 'communication_month', 'communication_day',
       'last_call_outcome']]

plt.figure(figsize=(40,45), facecolor='white')

plotnumber = 1

for column in data1:
    if plotnumber <= 16:
        plt.subplot(5, 2, plotnumber)
        sns.histplot(x=data1[column], hue=data['y'])
        plt.xlabel(column, fontsize=30)  # Decreased the fontsize for better visualization
        plt.ylabel("y", fontsize=20, rotation=90)
        plt.xticks(rotation=45)
    plotnumber += 1

plt.tight_layout()

plt.figure(figsize=(40,45), facecolor='white')

plotnumber = 1

for column in data2:
    if plotnumber <= 16:
        plt.subplot(5, 2, plotnumber)
        sns.countplot(x=data2[column], hue=data['y'])
        plt.xlabel(column, fontsize=30)  # Decreased the fontsize for better visualization
        plt.ylabel("y", fontsize=20, rotation=90)
        plt.xticks(rotation=45)
    plotnumber += 1

plt.tight_layout()

sns.relplot(x='duration',y='campaign',col='y',data=data)
plt.show()

sns.pairplot(data)

for i in data.columns:
    data[i].replace("", np.nan,inplace=True)
data.isnull().sum()

from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()

#job
data.job =enc.fit_transform(data.job )

#marital
data.marital =enc.fit_transform(data.marital )

#education
data.education =enc.fit_transform(data.education )

#default
data.default =enc.fit_transform(data.default )

#housing
data.housing =enc.fit_transform(data.housing )

#personal_loan
data.personal_loan =enc.fit_transform(data.personal_loan )

#communication_means
data.communication_means =enc.fit_transform(data.communication_means )

#communication_month
data.communication_month =enc.fit_transform(data.communication_month )

#communication_day
data.communication_day =enc.fit_transform(data.communication_day )

#last_call_outcome
data.last_call_outcome =enc.fit_transform(data.last_call_outcome )

#y
data.y =enc.fit_transform(data.y )

data['emp_var_rate'] = data['emp_var_rate'].astype(int)
data['cons.price.idx'] = data['cons.price.idx'].astype(int)
data['cons.conf.idx'] = data['cons.conf.idx'].astype(int)
data['euribor3m'] = data['euribor3m'].astype(int)

corr_data=data[['age', 'job', 'marital', 'education', 'default', 'housing',
       'personal_loan', 'communication_means', 'communication_month',
       'communication_day', 'duration', 'campaign', 'last_contacted',
       'how_many_times_contacted', 'last_call_outcome', 'emp_var_rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']]

plt.figure(figsize=(23,25))
sns.heatmap(corr_data.corr(),annot=True)


plt.figure(figsize=(14, 14))
plotnumber = 1

for i in data:
    plt.subplot(5, 5, plotnumber)
    sns.boxplot(x=data[i])
    plotnumber += 1
    if plotnumber > 25:
        break

plt.tight_layout()
plt.show()

X=data.drop('y',axis=1)
y=data.y

X.shape

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)

X_train
X_test
y_train
y_test

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Let's see oue target variable is balanced or not if it's not balanced. So, we have to apply SMOTE for balanced the data.
y.value_counts()
# here we can see our target variable is not balanced.

# Importing necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


# Initialize Logistic Regression Classifier
logreg_classifier = LogisticRegression(max_iter=1000, random_state=42)

# Train the classifier on the training data
logreg_classifier.fit(X_train, y_train)

# Predict labels for the test set
y_pred = logreg_classifier.predict(X_test)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score


# Define DecisionTreeClassifier model
model = DecisionTreeClassifier()

# Train the model
model.fit(X_train, y_train)

# Predict using the trained model
y_pred = model.predict(X_test)

# Calculate F1 score on test set
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 score on test set:", f1)

# Importing necessary libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score


# Define the hyperparameters grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Initialize Support Vector Classifier
svm_classifier = SVC()

# Initialize GridSearchCV with the defined hyperparameters grid and scoring metric
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Use the best estimator to predict labels for the test set
best_svm_classifier = grid_search.best_estimator_
y_pred = best_svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# Calculate F1 score using the best estimator
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# X = Your feature matrix
# y = Your target vector


# Initialize SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)

# Train the classifier on the training data
svm_classifier.fit(X_train, y_train)

# Predict labels for the test set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

# Importing necessary libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score


# Define the hyperparameters grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Initialize Support Vector Classifier
svm_classifier = SVC()

# Initialize GridSearchCV with the defined hyperparameters grid and scoring metric
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Use the best estimator to predict labels for the test set
best_svm_classifier = grid_search.best_estimator_
y_pred = best_svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score using the best estimator
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


# Define RandomForestClassifier model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

# Predict using the trained model
y_pred = model.predict(X_test)

# Calculate F1 score on test set
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 score on test set:", f1)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define RandomForestClassifier model
model = RandomForestClassifier()

# Define hyperparameters grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Define grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)

# Perform grid search
grid_result = grid_search.fit(X_train, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Predict using the best model
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate F1 score on test set
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 score on test set:", f1)

# Importing necessary libraries
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score


# Initialize Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(random_state=42)

# Train the classifier on the training data
gb_classifier.fit(X_train, y_train)

# Predict labels for the test set
y_pred = gb_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score


# Define GradientBoostingClassifier model
model = GradientBoostingClassifier()

# Define hyperparameters grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

# Define grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)

# Perform grid search
grid_result = grid_search.fit(X_train, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Predict using the best model
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate F1 score on test set
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 score on test set:", f1)

